# Adding Custom Variables

If you tried to run the configuration from the last tutorial, we saw two problems.

1. `elemental_carbon` and `organic_carbon` are not known AeroTools variables
2. `elemental_carbon` and `organic_carbon` could not be read by the EMEP reader

This is of course a problem made on purpose. But by using PyAeroval to solve this problem, we will see how this could be done in general.

## Have variables are available in PyAerocom?

Sadly there aren't any easy wait to check which variables are available in PyAerocom. Through [the observation explorer](https://aeroval.met.no/pages/evaluation/?project=obs&experiment=meteorology) we can look at the most used variables, and which networks contain them. But this does not include everything.

To find a list of all supported variables we need to got to github and find [variables.ini](https://github.com/metno/pyaerocom/blob/main-dev/pyaerocom/data/variables.ini). This does not describe from which variables is supported by which network or model. Below we will look at how to add extra variables to this list during runtime.

If you want to see which variables are supported for reading from EMEP, we can look at [this file of github](https://github.com/metno/pyaerocom/blob/main-dev/pyaerocom/io/mscw_ctm/emep_variables.toml). This file maps Aerocom names to EMEP names. We will look at how to add an extra mapping in our config, as well as using already existing mappings to make new variables.

## Adding an extra variable

Here we will look at how to add an extra variable, in the same way as is done in `variables.ini`.

PyAerocom has a static object called `config`, which contains a function for adding new variables

In [2]:
from pyaerocom.variable import Variable
from pyaerocom import const

variables = {
    "elemental_carbon": Variable(var_name="elementalcarbon", units="ug Cm-3"),
    "organic_carbon": Variable(var_name="organiccarbon", units="ug Cm-3"),
}

const.register_custom_variables(variables)

The function `register_custom_variables` takes a dictionary there the vales are `Variable` objects. These objects need at least the variable name and unit. It can also take other arguments, such as `min` and `max`. In `variables.ini` you can see all the other arguments that are used.

## Adding extra variables to EMEP

For EMEP we need to add our extra variables to the configuration. We will use two different methods for our two different types of carbon.

We'll start with organic carbon. Here we will simply map our new variable to an existing EMEP variable `SURF_ugC_PM_OM25`.

In [3]:
folder_EMEP = "/lustre/storeB/project/fou/kl/emep/People/danielh/projects/pyaerocom/workshop/emep/mod/2020"

EMEP = dict(
        model_id="EMEP",
        model_data_dir=folder_EMEP,
        gridded_reader_id={"model": "ReadMscwCtm"}, # Tells pyaerocom to use the EMEP reader instead of the default aerocom reader

        emep_vars={"organiccarbon": "SURF_ugC_PM_OM25"},
    )

This is a new and updated version of our model entry in the PyAeroval config. At the bottom line we can see the line `emep_vars={"organiccarbon": "SURF_ugC_PM_OM25"},`. This tells the reader which EMEP variables to read for our organic carbon.

For organic carbon we will use some already existing variables, namely `concecCoarse` and `concecFine`. We will add these two variables to make total elemental carbon

In [ ]:
def calc_elecmentalcarbon(concecCoarse, concecFine):

    elementalcarbon = concecCoarse.copy(deep=True) + concecFine.copy(
        deep=True
    )  # Adds the two variables
    elementalcarbon.attrs["units"] = "ug C m-3"  # Make sure the unit is correct
    return elementalcarbon


folder_EMEP = "/lustre/storeB/project/fou/kl/emep/People/danielh/projects/pyaerocom/workshop/emep/mod/2017"

EMEP = dict(
    model_id="EMEP",
    model_data_dir=folder_EMEP,
    gridded_reader_id={
        "model": "ReadMscwCtm"
    },  # Tells pyaerocom to use the EMEP reader instead of the default aerocom reader
    emep_vars={"organiccarbon": "SURF_ugC_PM_OM25"},
    model_read_aux={
        "elementalcarbon": {
            "vars_required": ["concecCoarse", "concecFine"],
            "fun": calc_elecmentalcarbon,
        }
    },
)

This method is a bit more complicated. The first thing we do is to make a function which tells the EMEP reader how to combine our exiting variables. This is called `calc_elecmentalcarbon` here. It takes our variables, adds them, hardcodes the unit and then returns the result. In our model config, we can now see an extra entry `model_read_aux`, which tells the reader which variables to read, and which function it needs to use on them. Here we see that we need to read `concecCoarse` and `concecFine`, and then use our new function to calculate `elementalcarbon`. 

Note that we can use as many existing variables as we want, and make the function as easy or complicated as we want. We can also use newly defined variables, such as `organiccarbon`.

In the folder for this tutorial you can find a new config, which can be used to run the experiment with our added variables.

## Best Practice

Note that this way of adding custom variables should only be used for testing, and not in production. Meaning that when you have found all the variables you want to add to AeroTools (Pyaerocom and/or the EMEP reader), you should make an issue on [the PyAerocom github](https://github.com/metno/pyaerocom) or ask one of the developers to add it. There are couple of reasons why

1. It's it not as easy to reproduce experiments where you have added custom variables, since your additions are not documented on the webpage or in the PyAerocom code.
2. You might get errors/memory errors which are difficult to handle, especially with the EMEP reader
3. Other parts having to do with the variables are hardcoded in PyAerocom/PyAeroval. Examples are axis scaling, display names, etc.